In [14]:
'''MultiHash PCY algorithm from the scratch ---> implementation on finding Asscociation Rules by using pySpark framework '''
import os
import sys
import findspark
findspark.init()
from pyspark import SparkContext,SparkConf,AccumulatorParam
conf=SparkConf()
sc = SparkContext(conf=conf ,master="local[*]")
text = sc.textFile("browsing.txt")
text=text.map(lambda x:x.strip().split(' '))
import random
import math
'''subclass the AccumulatroParam helper object and overwrite the addInPlace method'''
class DictAccumulatorParam(AccumulatorParam):
    def zero(self, initialValue):
        return initialValue
    def addInPlace(self, v1, v2):
        for x in v2.keys():
            if x in v1.keys():
                v1[x]+=v2[x]
            else:
                v1[x]=v2[x]
        return v1
    
'''create the shared dict which is used on each worker node'''    
ac1=sc.accumulator(dict(), DictAccumulatorParam())
ac2=sc.accumulator(dict(), DictAccumulatorParam())

'''hash each item on one bucket from each sample and increment by one the coresponding value '''
a = random.randint(100,1000)
b = random.randint(100,1000)
support = 100
mod_hash = 31201
def create_2_dicts(it):
    d1=dict()
    d2=dict()
    hash_key_1 = a*hash(it)%mod_hash
    if hash_key_1 in d1.keys():
        d1[hash_key_1]+=1
    else:
        d1[hash_key_1]=1
    hash_key_2 = b*hash(it)%mod_hash
    if hash_key_2 in d2.keys():
        d2[hash_key_2]+=1
    else:
        d2[hash_key_2]=1
    ac1.add(d1)
    ac2.add(d2)
    
def create_pairs(x):
    temp=[]
    for i in range(len(x)):
        for j in range(i+1,len(x)):
            temp.append(tuple(sorted((x[i],x[j]))))
    return temp

count=text.flatMap(lambda x:x).map(lambda x:(x,1)).countByKey()
text=text.map(lambda x:create_pairs(x))
'''
d11=dict()
d22=dict()
text_ = text.collect()
for x in text_:
    for it in x:
        hash_key_1 = a*hash(it)%31201
        if hash_key_1 in d11.keys():
            d11[hash_key_1]+=1
        else:
            d11[hash_key_1]=1
        hash_key_2 = b*hash(it)%31201
        if hash_key_2 in d22.keys():
            d22[hash_key_2]+=1
        else:
            d22[hash_key_2]=1
'''
count_d1_d2 = text.flatMap(lambda x:x).foreach(lambda x:create_2_dicts(x))
d1=ac1.value#sc.broadcast(ac1.value)
d2=ac2.value#sc.broadcast(ac2.value)

#print(max(list(d1.values())),"---->",max(list(d2.values())))
print("Number of frequent singleton items --->",len(count))

#count = sc.broadcast(count)
def filtering_itemset(x):
    temp=[]
    for i in x:
        key_1 = a*hash(i)%mod_hash
        key_2 = b*hash(i)%mod_hash
        if key_1 in d1.keys() and key_2 in d2.keys():
            if d1[key_1]>=support and d2[key_2]>=support and count[i[0]]>=support and count[i[1]]>=support:
                temp.append(i)
    return temp

text=text.map(lambda x: filtering_itemset(x)).filter(lambda x:x!=[])

#text=list(map(lambda x:filtering_itemset(x,d1,d2,count), text.collect()))
#text=sc.paralellize(text)
'''
text = text.collect()
for i in range(len(text)):
    temp = []
    for i in text[i]:
        if i in d1.keys() and i in d2.keys():
            if d1[i].value>=100 and d2[i].value>=100 and count[i[0]].value>=100 and count[i[1]].value>=100:
                temp.append(i)
    text[i] = temp
text=sc.paralellize(text)
'''
print('Pairs ', text.take(1))
'''computer the counts of frequent pairs and compute condidence along with Association rules '''
freq=text.flatMap(lambda x:x).map(lambda x:(x,1)).reduceByKey(lambda x,y:x+y).filter(lambda x:x[1]>=support)
print("Number of frequent pair items --->",freq.count())

def compute_confidence(x,countIDs):
    temp=[]
    for i in range(len(x[0])):
        temp.append(x[1]/countIDs[x[0][i]])
    idx=temp.index(max(temp))
    ret=()
    for i in range(len(x[0])):
        if i!=idx:
            ret+=(x[0][i],)
    return ret+(x[0][idx],max(temp))

Association_Rule_1=freq.map(lambda x:compute_confidence(x,count))

print("Association Rules for pair frequent Itemsets with Confedence>0.95)",sorted(Association_Rule_1.filter(lambda x:x[2]>0.95).take(8)))

'''Find frequend tripletes of items'''
print('Pairs', text.take(1))
def create_triplets(l):
    temp=[]
    for i in range(len(l)):
        for j in range(i+1,len(l)):
            for z in l[j]:
                if z not in l[i]:
                    t=tuple(sorted(l[i]+(z,)))
                    if t not in temp and tuple(sorted(l[i])) in count1.keys():
                        temp.append(t)
    i=0
    while(i<len(temp)):
        flag=True
        for j in range(0,len(temp[i])):
            for k in range(j+1,len(temp[i])):
                if tuple(sorted((temp[i][j],temp[i][k]))) not in l:
                    del temp[i]
                    flag=False
                    break
            if flag==False:
                i=i-1
                break
        i+=1
    return temp

count1=text.flatMap(lambda x:x).map(lambda x: tuple(sorted(x))).map(lambda x:(x,1)).countByKey()
text=text.map(lambda x:create_triplets(x))

ac1=sc.accumulator(dict(), DictAccumulatorParam())
ac2=sc.accumulator(dict(), DictAccumulatorParam())
count_d1_d2=text.flatMap(lambda x:x).foreach(lambda x:create_2_dicts(x))

d1=ac1.value#sc.broadcast(ac1.value)
d2=ac2.value#sc.broadcast(ac2.value)
#count1=freq.collectAsMap()

def check(x):
    for i in range(len(x)):
        for j in range(i+1,len(x)):
            temp = tuple(sorted((x[i],x[j])))
            if temp in count1.keys():
                if count1[temp] < support:
                    return False
    return True
    
def filtering_itemset1(x):
    temp=[]
    for i in x:
        key_1 = a*hash(i)%mod_hash
        key_2 = b*hash(i)%mod_hash
        if key_1 in d1.keys() and key_2 in d2.keys():
            if d1[key_1]>=support and d2[key_2]>=support and check(i) == True:
                temp.append(i)
    return temp

text=text.map(lambda x: filtering_itemset1(x)).filter(lambda x:x!=[])
print('Triplets',text.take(1))

'''computer the counts of frequent triples and compute condidence along with Association rules '''
freq=text.flatMap(lambda x:x).map(lambda x:(x,1)).reduceByKey(lambda x,y:x+y).filter(lambda x:x[1]>=support)
print("Number of frequent triple items --->",freq.count())
#tempFile = NamedTemporaryFile(delete=True)
#tempFile.close()
def compute_triplet_confidence(x,countIDs,countIDs1):
    temp=[]
    for i in range(len(x[0])-1):
        temp.append(((x[0][i],),countIDs[x[0][i]]))
        for j in range(i+1,len(x[0])):
                    key=tuple(sorted((x[0][i],x[0][j])))
                    temp.append((key,countIDs1[key]))
    I,support=min(temp,key=lambda x:x[1])
    j=()
    for i in x[0]:
        if i not in I:
            j+=(i,)
    confidence=x[1]/support
    if len(j)==1:
        probability=countIDs[j[0]]/length
    else:
        probability=countIDs1[j]/length
    return I+('--->',j,confidence,math.fabs(confidence-probability))

#sc.broadcast
Association_Rule_2=freq.map(lambda x:compute_triplet_confidence(x,count,count1))
#print(type(Association_Rule_2))
print("Association for three pair frequent Itemsets Rule with Conf>0.55)",sorted(Association_Rule_2.filter(lambda x:x[3]>=0.55).take(20)))
#Association_Rule_2.saveAsTextFile('temp.txt')
sc.stop()

Number of frequent singleton items ---> 12592
Pairs  [[('ELE17451', 'FRO11987'), ('ELE17451', 'SNA90258'), ('ELE17451', 'GRO99222'), ('GRO99222', 'SNA90258')]]
Number of frequent pair items ---> 1334
Association Rules for pair frequent Itemsets with Confedence>0.95) [('DAI62779', 'DAI23334', 0.9545454545454546), ('FRO40251', 'DAI88079', 0.9867256637168141), ('FRO40251', 'DAI93865', 1.0), ('FRO40251', 'ELE12951', 0.9905660377358491), ('FRO40251', 'FRO92469', 0.983510011778563), ('FRO40251', 'GRO38636', 0.9906542056074766), ('FRO40251', 'GRO85051', 0.999176276771005), ('SNA82528', 'DAI43868', 0.972972972972973)]
Pairs [[('ELE17451', 'FRO11987'), ('ELE17451', 'SNA90258'), ('ELE17451', 'GRO99222'), ('GRO99222', 'SNA90258')]]
Triplets [[('DAI75645', 'GRO56726', 'GRO73461')]]
Number of frequent triple items ---> 0
Association for three pair frequent Itemsets Rule with Conf>0.55) []


In [12]:
len(d11.keys()), len(d1.keys())

(31201, 31201)

In [13]:
all(list(d11.keys())) in list(d1.keys())

True

In [14]:
key = list(d11.keys())[0]
key

25337

In [16]:
d1[key],d11[key]

(69, 58)

In [18]:
type(text)

list